In [5]:
from __future__ import print_function # so print doesn't show brackets
import numpy as np
import itertools as itr

import os as os
import sys as sys 
import pandas as pd
import warnings
import time as time
import random
import pickle
pickle.HIGHEST_PROTOCOL = 2
sys.path.append(os.path.join("..", "Libraries","QML_lib"))
from RedisSettings import *
import Evo as evo
import DataBase 
from QMD import QMD #  class moved to QMD in Library
import QML
import ModelGeneration 
import BayesF
import matplotlib.pyplot as plt
#from pympler import asizeof
import matplotlib.pyplot as plt
paulis = ['x', 'y', 'z'] # will be chosen at random. or uncomment below and comment within loop to hard-set

import time as time 
import argparse

parser = argparse.ArgumentParser(description='Pass variables for (I)QLE.')

import warnings
warnings.filterwarnings("ignore")


num_runs=1
num_particles= 40
num_experiments = 10
num_times_bayes = 5
qle=True


import time
start = time.time()

initial_op_list = ['xTi', 'yTi', 'zTi']
#initial_op_list = ['x', 'y', 'z']

num_ops = len(initial_op_list)
for i in range(num_runs):
    print("\ni=",i)
    print(num_particles, "Paricles for ", num_experiments, "Experiments:")
    true_op = 'xTiPPyTiPPzTiPPxTxPPyTyPPzTz'
#    true_params = [np.random.rand()]
    true_params = [0.19, 0.21, 0.8, 0.22, 0.20, 0.27]
    qmd = QMD(
        initial_op_list=initial_op_list, 
        true_operator=true_op, 
        true_param_list=true_params, 
        num_particles=num_particles,
        num_experiments = num_experiments, 
        num_times_for_bayes_updates = num_times_bayes,
        qle=qle,
        num_probes=5,
        gaussian=False, 
        max_num_branches = 0,
        max_num_qubits = 10, 
        parallel = True,
        use_exp_custom=False, 
        compare_linalg_exp_tol=1e-20,
        growth_generator='ising_non_transverse'
        #growth_generator='experimental_qmd'
    )
   # qmd.learnModelNameList(model_name_list=['z'], blocking=True, use_rq=False)
    qmd.runRemoteQMD(num_spawns=3)
    
end = time.time()
print("\n\nTime taken:", end-start)



i= 0
40 Paricles for  10 Experiments:

Running  QLE  for true operator  xTiPPyTiPPzTiPPxTxPPyTyPPzTz  with parameters :  [0.19, 0.21, 0.8, 0.22, 0.2, 0.27]
Model  xTi  not previously considered -- adding.
Initialization Ready
Model  yTi  not previously considered -- adding.
Initialization Ready
Model  zTi  not previously considered -- adding.
Initialization Ready
Model  xTi being learned
QHL for xTi remote: True
Trying to load qmd info from redis db
Trying to load probe dict info from redis db
QMD info loaded
Initialization Ready
QML instance generated
Initial time selected > 2.45265687468
Results for QHL on  xTi
Final time selected > 8.17670925064
Final Parameters mean and stdev (term  xTi ):[ 0.89440738  0.15378918]
Model xTi learned and pickled to redis DB.
Model xTi added to queue.
Model  yTi being learned
QHL for yTi remote: True
Trying to load qmd info from redis db
Trying to load probe dict info from redis db
QMD info loaded
Initialization Ready
QML instance generated
Initial t

Results for QHL on  yTiPPxTiPPyTiPPxTx
Final time selected > 2.27554625476
Final Parameters mean and stdev (term  yTi ):[ 0.18751301  0.45041569]
Final Parameters mean and stdev (term  xTi ):[ 0.85120444  0.39589956]
Final Parameters mean and stdev (term  yTi ):[ 0.20031289  0.52633141]
Final Parameters mean and stdev (term  xTx ):[ 0.40093412  0.4812509 ]
Model yTiPPxTiPPyTiPPxTx learned and pickled to redis DB.
Model yTiPPxTiPPyTiPPxTx added to queue.
Model  yTiPPxTiPPyTiPPyTy being learned
QHL for yTiPPxTiPPyTiPPyTy remote: True
Trying to load qmd info from redis db
Trying to load probe dict info from redis db
QMD info loaded
Initialization Ready
QML instance generated
Initial time selected > 0.292263433212
Results for QHL on  yTiPPxTiPPyTiPPyTy
Final time selected > 0.32220729304
Final Parameters mean and stdev (term  yTi ):[ 0.50356685  0.60481018]
Final Parameters mean and stdev (term  xTi ):[ 0.50741879  0.54266433]
Final Parameters mean and stdev (term  yTi ):[ 0.56380456  0.61

Results for QHL on  yTiPPxTiPPyTiPPzTzPPxTxPPyTy
Final time selected > 0.229808981995
Final Parameters mean and stdev (term  yTi ):[ 0.42690822  0.60066492]
Final Parameters mean and stdev (term  xTi ):[ 0.67465469  0.542917  ]
Final Parameters mean and stdev (term  yTi ):[ 0.47478654  0.55716566]
Final Parameters mean and stdev (term  zTz ):[ 0.65945842  0.52567845]
Final Parameters mean and stdev (term  xTx ):[ 0.52628195  0.55405104]
Final Parameters mean and stdev (term  yTy ):[ 0.48796976  0.59694421]
Model yTiPPxTiPPyTiPPzTzPPxTxPPyTy learned and pickled to redis DB.
Model yTiPPxTiPPyTiPPzTzPPxTxPPyTy added to queue.
Model  yTiPPxTiPPyTiPPzTzPPxTxPPzTz being learned
QHL for yTiPPxTiPPyTiPPzTzPPxTxPPzTz remote: True
Trying to load qmd info from redis db
Trying to load probe dict info from redis db
QMD info loaded
Initialization Ready
QML instance generated
Initial time selected > 0.369290365469
Results for QHL on  yTiPPxTiPPyTiPPzTzPPxTxPPzTz
Final time selected > 0.616884375746
F

In [6]:
import ModelGeneration as md

In [10]:
mod  = 'yTiPPxTiPPyTiPPzTzPPxTxPPxTx'

for spawn_no in [6,7,8]:
    mod_list = [mod]
    new_models=md.ising_transverse(mod_list, spawn_step=6)
    print("step:", spawn_no, "New models:\n", new_models)
    mod=new_models[0]

step: 6 New models:
 ['yTiPPxTiPPyTiPPzTzPPxTxPPxTxPPxTy', 'yTiPPxTiPPyTiPPzTzPPxTxPPxTxPPxTz', 'yTiPPxTiPPyTiPPzTzPPxTxPPxTxPPyTx', 'yTiPPxTiPPyTiPPzTzPPxTxPPxTxPPyTz', 'yTiPPxTiPPyTiPPzTzPPxTxPPxTxPPzTy']
step: 7 New models:
 ['yTiPPxTiPPyTiPPzTzPPxTxPPxTxPPxTyPPxTz', 'yTiPPxTiPPyTiPPzTzPPxTxPPxTxPPxTyPPyTx', 'yTiPPxTiPPyTiPPzTzPPxTxPPxTxPPxTyPPyTz', 'yTiPPxTiPPyTiPPzTzPPxTxPPxTxPPxTyPPzTy']
step: 8 New models:
 ['yTiPPxTiPPyTiPPzTzPPxTxPPxTxPPxTyPPxTzPPyTx', 'yTiPPxTiPPyTiPPzTzPPxTxPPxTxPPxTyPPxTzPPyTz', 'yTiPPxTiPPyTiPPzTzPPxTxPPxTxPPxTyPPxTzPPzTy']


In [23]:
def ising_non_transverse(model_list, spawn_step):
    single_qubit_terms = ['xTi', 'yTi', 'zTi']
    nontransverse_terms = ['xTx', 'yTy', 'zTz']

    if len(model_list) > 1:
        print("Only one model required for non-transverse Ising growth.")
        return False
    else:
        model = model_list[0]
    
    present_terms = model.split('PP')
    print("Present:", present_terms)
    new_models = []
    if spawn_step in [1,2]:
        for term in single_qubit_terms:
            if term not in present_terms:
                new_model = model+'PP'+term
                new_models.append(new_model)

    elif spawn_step in [3,4,5]:
        for term in nontransverse_terms:
            if term not in present_terms:
                new_model = model+'PP'+term
                new_models.append(new_model)
                
    return new_models    


In [22]:
'xTi'.split('PP')

['xTi']

In [7]:
def hyperfine_like(model_list, spawn_step):
# TODO before using this function, need to add a max_spawn_depth to the dict above for ising_transverse. How many spawns can it support?
    import random
    single_qubit_terms = ['xTi', 'yTi', 'zTi']
    nontransverse_terms = ['xTx', 'yTy', 'zTz']
    transverse_terms = ['xTy', 'xTz', 'yTx', 'yTz', 'zTy', 'zTx']
    all_two_qubit_terms =  single_qubit_terms + nontransverse_terms  + transverse_terms
    if len(model_list) > 1:
        print("Only one model required for transverse Ising growth.")
        return False
    else:
        model = model_list[0]

    present_terms = model.split('PP')

    new_models = []
    if spawn_step in [1,2]:
        for term in single_qubit_terms:
            if term not in present_terms:
                new_model = model+'PP'+term
                new_models.append(new_model)

    else: 
        
        for i in range(3):
            term = random.choice(all_two_qubit_terms)
            
            if term not in present_terms:
                new_model = model+'PP'+term
                new_models.append(new_model)
                
        
    """
    elif spawn_step in [3,4,5]:
        for term in nontransverse_terms:
            if term not in present_terms:
                new_model = model+'PP'+term
                new_models.append(new_model)

    else:
        for term in transverse_terms:
            if term not in present_terms:
                new_model = model+'PP'+term
                new_models.append(new_model)
    """
    return new_models    


In [8]:
mod  = 'xTi'

for spawn_no in range(1,9):
    mod_list = [mod]
    print("input model:", mod_list[0])
    new_models=ising_transverse(mod_list, spawn_step=spawn_no)
    print("step:", spawn_no, "New models:\n", new_models)
    mod=new_models[0]

input model: xTi
step: 1 New models:
 ['xTiPPyTi', 'xTiPPzTi']
input model: xTiPPyTi
step: 2 New models:
 ['xTiPPyTiPPzTi']
input model: xTiPPyTiPPzTi
step: 3 New models:
 ['xTiPPyTiPPzTiPPxTz', 'xTiPPyTiPPzTiPPyTy', 'xTiPPyTiPPzTiPPzTy']
input model: xTiPPyTiPPzTiPPxTz
step: 4 New models:
 ['xTiPPyTiPPzTiPPxTzPPzTx', 'xTiPPyTiPPzTiPPxTzPPzTx']
input model: xTiPPyTiPPzTiPPxTzPPzTx
step: 5 New models:
 ['xTiPPyTiPPzTiPPxTzPPzTxPPyTz', 'xTiPPyTiPPzTiPPxTzPPzTxPPzTy', 'xTiPPyTiPPzTiPPxTzPPzTxPPyTx']
input model: xTiPPyTiPPzTiPPxTzPPzTxPPyTz
step: 6 New models:
 ['xTiPPyTiPPzTiPPxTzPPzTxPPyTzPPzTy']
input model: xTiPPyTiPPzTiPPxTzPPzTxPPyTzPPzTy
step: 7 New models:
 ['xTiPPyTiPPzTiPPxTzPPzTxPPyTzPPzTyPPzTz', 'xTiPPyTiPPzTiPPxTzPPzTxPPyTzPPzTyPPzTz']
input model: xTiPPyTiPPzTiPPxTzPPzTxPPyTzPPzTyPPzTz
step: 8 New models:
 ['xTiPPyTiPPzTiPPxTzPPzTxPPyTzPPzTyPPzTzPPyTy', 'xTiPPyTiPPzTiPPxTzPPzTxPPyTzPPzTyPPzTzPPxTy']
